In [1]:
import sys, os
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print("Added to sys.path:/", repo_root)
from fixedincomelib import *
print("Fixed Income Library is loaded.")

Added to sys.path:/ /Users/lunli/Documents/FixedIncomeLib
Fixed Income Library is loaded.


### Create Build Method Collection

In [2]:
bm_list = []
# create yield curve build method
content_sofr = {
    'TARGET' : 'SOFR-1B',
    'OVERNIGHT INDEX FUTURE' : 'SOFR-FUTURE-3M'}
bm_list.append(qfCreateBuildMethod('YIELD_CURVE', content_sofr))
# create yield curve common build method 
content = {
    'TARGET' : 'USD',
    'FUNDING PARAMETERS' : 'USD-FUNDING-PARAMETERS',
    'SOLVER' : 'BRENTQ'}
bm_list.append(qfCreateBuildMethod('YIELD_CURVE_COMMON', content))
build_method_collection = qfCreateModelBuildMethodCollection(bm_list)

### Create Data Collection

In [3]:
data_list = []
df = pd.DataFrame(index=['2026-01-15x2026-04-15', '2026-04-15x2026-07-15'])
df['values'] = [96.50, 97.50]
data_type = 'OVERNIGHT INDEX FUTURE'
data_conv = 'SOFR-FUTURE-3M'
data_list.append(qfCreateData1D(data_type, data_conv, df))
df_dg = pd.DataFrame([['Overnight Index Future', 'SOFR-FUTURE-3M', 'SOFR-1B']], columns=['DATA TYPE', 'DATA CONVENTION', 'FUNDING IDENTIFIER'])
data_type = 'DATA GENERIC'
data_label = 'USD-FUNDING-PARAMETERS'
data_list.append(qfCreateDataGeneric(data_type, data_label, df_dg))
data_collection = qfCreateDataCollection(data_list)

### Create Model Yield Curve

In [4]:
value_date = '2025-12-23'
yc_model = qfCreateModel(value_date, 'YIELD_CURVE', data_collection, build_method_collection)

### Re-Pricing Calibration Instruments

In [5]:
### create calibration instruments
calib_prod_1 = qfCreateProductFromDataConvention(value_date, data_conv, df.index[0], df.values[0][0])
calib_prod_2 = qfCreateProductFromDataConvention(value_date, data_conv, df.index[1], df.values[1][0])
### create valuation parameters collection
data_type ='OVERNIGHT INDEX FUTURE'
v = df_dg[(df_dg['DATA TYPE'].str.upper() == data_type) & (df_dg['DATA CONVENTION'].str.upper() == content_sofr[data_type])]
fi = v['FUNDING IDENTIFIER'].values[0]
fi_vp = qfCreateValuationParameters('FUNDING INDEX PARAMETER', {'Funding Index' : fi})
vp_collection = qfCreateValuationParametersCollection([fi_vp])
### valuation
fut_price_1 = 100 - 100 * qfCreateValueReport(yc_model, calib_prod_1, vp_collection, 'parrateorspread')
print(f'market level is {df.values[0][0]}, and benchmark is {fut_price_1}.')
fut_price_2 = 100 - 100 * qfCreateValueReport(yc_model, calib_prod_2, vp_collection, 'parrateorspread')
print(f'market level is {df.values[1][0]}, and benchmark is {fut_price_2}.')

market level is 96.5, and benchmark is 96.50000000000007.
market level is 97.5, and benchmark is 97.49999999999996.


In [6]:
# display(qfCreateValueReport(yc_model, calib_prod_1, vp_collection, 'firstorderrisk'))
display(qfCreateValueReport(yc_model, calib_prod_2, vp_collection, 'firstorderrisk'))

array([     0., 250000.])